# Модель GARCH(1,1) Skewed-t

- Обучение на returns
- Прогноз волатильности (скользящее окно)
- Tail risk

Шаблон содержит структуру для быстрого запуска, визуализирует сходится ли модель.


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Попытка импорта arch для GARCH
try:
    from arch import arch_model
    print("✅ Библиотеки загружены (arch доступен)")
    ARCH_AVAILABLE = True
except ImportError:
    print("⚠️ Библиотека 'arch' не установлена. Установите: pip install arch")
    ARCH_AVAILABLE = False


## Загрузка данных


In [ ]:
DATA_DIR = Path('data') / 'features'
ticker = 'SBER'
df = pd.read_parquet(DATA_DIR / f"{ticker}_with_targets.parquet")

# Подготовка данных для GARCH
returns = df['log_return'].dropna() * 100  # Масштабируем для стабильности
print(f"✅ Данные загружены: {len(returns)} наблюдений")
print(f"Статистика returns: mean={returns.mean():.4f}, std={returns.std():.4f}")


## Обучение GARCH(1,1) модели


In [ ]:
if ARCH_AVAILABLE:
    # Создаем и обучаем GARCH(1,1) модель с Skewed-t распределением
    model = arch_model(returns, vol='Garch', p=1, q=1, dist='skewt', rescale=False)
    
    print("Обучение GARCH(1,1) модели...")
    fitted_model = model.fit(disp='off', show_warning=False)
    
    print("\n" + "="*60)
    print(fitted_model.summary())
    print("="*60)
    
    # Извлечение условной волатильности
    conditional_vol = fitted_model.conditional_volatility
    
    # Прогноз волатильности на 1 шаг вперед
    forecast = fitted_model.forecast(horizon=1)
    forecast_variance = forecast.variance.iloc[-1, 0]
    forecast_vol = np.sqrt(forecast_variance)
    
    print(f"\n✅ Модель обучена")
    print(f"Прогноз волатильности на следующий день: {forecast_vol:.4f}%")
    
else:
    print("❌ GARCH модель недоступна. Установите библиотеку 'arch'.")


## Walk-forward прогнозирование


In [ ]:
if ARCH_AVAILABLE:
    # Walk-forward прогнозирование на последних 100 днях
    train_size = len(returns) - 100
    forecasts = []
    
    print("Walk-forward прогнозирование (может занять время)...")
    
    for i in range(train_size, len(returns)):
        train_data = returns.iloc[:i]
        
        # Обучаем модель на историческом окне
        model_temp = arch_model(train_data, vol='Garch', p=1, q=1, dist='skewt', rescale=False)
        res_temp = model_temp.fit(disp='off', show_warning=False, update_freq=0)
        
        # Прогноз на 1 шаг
        forecast_temp = res_temp.forecast(horizon=1)
        forecasts.append(np.sqrt(forecast_temp.variance.iloc[-1, 0]))
    
    # Добавляем прогнозы в DataFrame
    forecast_dates = df['date'].iloc[-100:]
    forecast_df = pd.DataFrame({
        'date': forecast_dates.values,
        'garch_forecast': forecasts
    })
    
    print(f"✅ Walk-forward прогнозирование завершено: {len(forecasts)} прогнозов")
    print(forecast_df.tail())
    
else:
    print("❌ Walk-forward недоступен без библиотеки 'arch'.")


## Сохранение результатов


In [ ]:
if ARCH_AVAILABLE:
    OUTPUT_DIR = Path('data') / 'models'
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    
    # Сохранение прогнозов
    output_path = OUTPUT_DIR / f"{ticker}_garch_forecasts.parquet"
    forecast_df.to_parquet(output_path, index=False)
    
    print(f"✅ Сохранено: {output_path}")
else:
    print("⚠️ Нечего сохранять без GARCH модели")
